In [7]:
#from pyscf import gto, scf, ci
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem 


In [3]:
symbols = ["Fe"]
coordinates = np.array([0.0, 0.0, 0.0])
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method='pyscf')

In [8]:
qubits

36

In [8]:
dataset_N2 = qml.data.load("qchem", molname="N2", bondlength=1.12, basis="STO-3G")
dataset_N2[0]

<Dataset = molname: N2, basis: STO-3G, bondlength: 1.12, attributes: ['basis', 'basis_rot_groupings', ...]>

In [9]:
dataset_N2[0].hamiltonian.wires

<Wires = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]>